# Welcome to my first jupyter notebook

In [5]:
import numpy as np
import sys 
import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pylab
import pygadgetreader as pg 
import scipy.integrate

These two functions are simply for calculating a_dot and 1 over that

In [6]:
def _a_dot(a, h0, om_m, om_l):
    om_k = 1.0 - om_m - om_l
    return h0 * a * np.sqrt(om_m * (a ** -3) + om_k * (a ** -2) + om_l)

In [7]:
def _a_dot_recip(*args):
    return 1. / _a_dot(*args)

Radpro pulls all the halos from the AHF halo files and finds the halo I'm interested in from the megerhist file. Ultimately it passes the data necessary to make radial density profile plots AND makes the vcirc vs rhalf/rmax plot.

In [8]:
def radpro(pathname,sname,hist,dmo,rhalf,rmax):
  global i, grain, clr, count,conv
  numcores = 16
  massp = 1.67262178e-24 #in grams
  gamma = 5./3.
  kb = 1.3806e-26 #in km^2*g/s^2 
  G = 4.3e-6 #in kpc/M_sun (km/s)^2   
  switch = 0
  for j in np.arange(numcores): #Bring together all the AHF files for the snapshot
    temph = glob.glob(pathname+'ahf_snap%03d/ahf.snap_%03d.%04d.z*.*.AHF_halos'%(i,i,j)) 
    temph = str(temph).strip('[]').replace("'","")
    h = np.genfromtxt(temph)
    temppr = glob.glob(pathname+'ahf_snap%03d/ahf.snap_%03d.%04d.z*.*.AHF_profiles'%(i,i,j))
    temppr = str(temppr).strip('[]').replace("'","")
    p = np.genfromtxt(temppr)
    if switch == 0:
      halo = h
      switch = 1
    if switch == 1:
      try:
        halo = np.vstack((halo,h))
      except:
        print "nothing there"
  for j in np.arange(len(halo)): #Find our halo from all the halos of the snapshot
    if halo[j,0] == hist[184-i,1]:
      mstar = halo[j,64]
      mvir = halo[j,3]
      vmax = halo[j,16]
      xcen = halo[j,5] #Grab relevant data (can always add more later)
      ycen = halo[j,6]
      zcen = halo[j,7]
      rvir = halo[j,11]/.71
      mass = pg.readsnap(sname, 'mass', 'dm')*1.e10/.71
      pos = pg.readsnap(sname, 'pos', 'dm')*conv
      tmp = np.array([0,0])
      diff = np.sqrt((pos[:,0]-xcen)**2+(pos[:,1]-ycen)**2+(pos[:,2]-zcen)**2)/.71
      mass = mass[diff<=rvir]
      diff = diff[diff<=rvir]
      cum = np.cumsum(mass[np.argsort(diff)])

      print xcen,ycen,zcen, np.sort(diff)
      binz = np.logspace(np.log10(.06),np.log10(rvir),grain)
      massall, bin_edge = np.histogram( diff,bins=binz, weights =mass) 
      x = 10**(np.log10(binz[1:])-np.log10(binz[1]/binz[0])/2)
      if dmo == 0:
        rmax = halo[j,12]/.71
        gmass = pg.readsnap(sname, 'mass', 'gas')*1.e10/.71
        smass = pg.readsnap(sname, 'mass', 'star')*1.e10/.71
        gpos = pg.readsnap(sname, 'pos', 'gas')*conv
        spos = pg.readsnap(sname, 'pos', 'star')*conv
        sdiff = np.sqrt((spos[:,0]-xcen)**2+(spos[:,1]-ycen)**2+(spos[:,2]-zcen)**2)/.71
        gdiff = np.sqrt((gpos[:,0]-xcen)**2+(gpos[:,1]-ycen)**2+(gpos[:,2]-zcen)**2)/.71
	smass = smass[sdiff<=rvir]
	gmass = gmass[gdiff<=rvir]
	sdiff = sdiff[sdiff<=rvir]
	gdiff = gdiff[gdiff<=rvir]
	scum = np.cumsum(smass[np.argsort(sdiff)])
	gcum = np.cumsum(gmass[np.argsort(gdiff)])
	totdiff = np.append(np.append(diff,sdiff),gdiff)
	totmass = np.append(np.append(mass,smass),gmass)
	totcum = np.cumsum(totmass[np.argsort(totdiff)])
	irhalf = abs(scum-sum(smass)/2).argmin()
	rhalf = sdiff[np.argsort(sdiff)][irhalf]
	itotrhalf = abs(np.sort(totdiff)-rhalf).argmin()
	print 'rhalf is %f'%rhalf
	irmax = np.sqrt(G*totcum/np.sort(totdiff)).argmax()
	rmax = totdiff[np.argsort(totdiff)][irmax]
	print 'rmax is %f'%rmax


	#print len(sdiff), binz, len(smass)
        gmassall, bin_edge = np.histogram( gdiff,bins=binz, weights =gmass) 
        smassall, bin_edge = np.histogram( sdiff,bins=binz, weights =smass)
	#irhalf = (abs(bin_edge-max(bin_edge[np.cumsum(smassall)<=sum(smassall)/2]))).argmin()

        #print bin_edge[np.cumsum(smassall)<=sum(smassall)/2],rvir,sum(smassall)#sum(smassall), np.cumsum(smassall) #x[irhalf],np.sqrt(G*sum(massall[:irhalf+1])/x[irhalf])
        ax1.scatter(rhalf,np.sqrt(G*totcum[itotrhalf]/rhalf),marker='s',s = 80,color='%s'%clr[count],label='%s'%(hnum))
        #ax1.scatter(rmax,np.sqrt(G*totcum[irmax]/rmax),marker='D',s = 80,color='%s'%clr[count],label='%s Rmax'%(hnum))
        return massall, gmassall, smassall, x, rhalf,rmax
      else:
        massall = 0.83120300751*massall
	irhalf = (abs(np.sort(diff)-rhalf)).argmin()
	irmax = (abs(np.sort(diff)-rmax)).argmin()
	print np.sqrt(G*cum[irhalf]/rhalf), np.sqrt(G*cum[irmax]/rmax)
	ax1.scatter(rhalf,np.sqrt(G*cum[irhalf]/rhalf),marker='^',s = 80,color='%s'%clr[count],label='%s DMO'%(hnum))
	#ax1.scatter(rmax,np.sqrt(G*cum[irmax]/rmax),marker='v',s = 80,color='%s'%clr[count],label='%s DMO Rmax'%(hnum))
        count +=1
        return massall, x

Plot radpro is what actually creates the radial density profiles

In [ ]:
def plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox):
  pylab.rcParams['xtick.major.pad']='6'
  pylab.rcParams['ytick.major.pad']='6'
  fig = plt.figure()
  ax = fig.add_subplot(111)
  hytot = hymass+hygmass+hysmass
  ax.loglog(hyx,hytot/(4*3.14159/3*hyx**3), 'k',linewidth=4,label = '11_13(ff) MFM')#(4*3.14159/3*hyx**3)
  #ax.loglog(x9_17,y9_17, 'b',linewidth=4,label = '9_17 MFM')
  #ax.loglog(x6_2,y6_2, 'g',linewidth=4,label = '6_2 MFM')
  #ax.loglog(x11_13,y11_13-g11_13-s11_13, 'b',linewidth=4,label = 'MFM DM')
  #ax.loglog(x11_13,g11_13, 'r',linewidth=4,label = 'MFM gas')
  #ax.loglog(x11_13,s11_13, 'y',linewidth=4,label = 'MFM star')
  #ax.loglog(dmox,dmoy, 'g--',linewidth=4, label = 'DMO (corrected)')
  ax.loglog(hyx,dmomass/(4*3.14159/3*hyx**3), 'r--',linewidth=4, label = 'DMO')
  ax.spines['bottom'].set_linewidth(4)
  ax.spines['top'].set_linewidth(4)
  ax.spines['right'].set_linewidth(4)
  ax.spines['left'].set_linewidth(4)
  ax.tick_params('both',length=5,width=2,which='minor')
  ax.tick_params('both',length=10,width=2,which='major')
  ax.xaxis.set_tick_params(labelsize=20)
  ax.yaxis.set_tick_params(labelsize=20)
  plt.xlabel('Radius (kpc)',fontsize=20, labelpad=-10)
  ax.xaxis.set_label_coords(.48,-.07)
  plt.ylabel(r'$\rho (M_\odot$/kpc$^3$)',fontsize=20, labelpad=-5)
  #plt.ylabel(r'$M_\odot/M_{DM,corrected}$',fontsize=20, labelpad=-5)
  plt.title('Halo %s Radial M(<r) (%d bins) Profile'%(hnum,grain))
  plt.xlim(.06,60)
  plt.ylim(1e4,1e9)
  #plt.ylim(1e-1,2e0)
  plt.legend(loc=3,prop={'size':10})
  #fname = 'm_enc_ratio_halo%s_%dbin_12_16_z0.pdf'%(hnum,grain)
  fname = 'radden_halo%s_%dbin_1_11_z0.pdf'%(hnum,grain)
  plt.savefig(fname,transparent=True)
  plt.show()
  plt.close()
  return True

Below are all the halos currently being plotted. 

In [9]:
grain = 60
i = 184
h0 = 71
om_l = 0.734
om_m = 0.266
conv = 3.085677581e+19
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
clr = ['r','b','k','g','y','m']
count = 0

In [10]:
conv = 1
hnum = '007'
res = '_11'
ver = '11_13'
pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
hymass,hygmass,hysmass,hyx,irhalf,rmax= radpro(pathname,sname,hist,0,0,0)

conv = 1000
pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

Returning DM    Mass
Returning DM    Positions
2294.59483773 2112.96809117 2323.62870453 [  8.72695906e-03   2.11494752e-02   2.70153337e-02 ...,   5.04083993e+01
   5.04084142e+01   5.04084419e+01]
Returning GAS   Mass
Returning STAR  Mass
Returning GAS   Positions
Returning STAR  Positions
rhalf is 0.832233
rmax is 7.963169
Returning DM    Mass
Returning DM    Positions
2305.25045329 2096.08853039 2302.48678561 [  1.42089117e-02   1.65638231e-02   1.91303020e-02 ...,   5.44084342e+01
   5.44084388e+01   5.44084479e+01]
27.2776972154 37.2999568751


In [11]:
hnum = '2'
res = '_13'
ver = '11_13'
pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
hymass,hygmass,hysmass,hyx,irhalf, rmax= radpro(pathname,sname,hist,0,0,0)

pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

Returning DM    Mass
Returning DM    Positions
12300.2891199 14698.8985763 11829.8025667 [  1.17761206e-02   2.07637543e-02   2.18972337e-02 ...,   5.25914193e+01
   5.25914679e+01   5.25914962e+01]
Returning GAS   Mass
Returning STAR  Mass
Returning GAS   Positions
Returning STAR  Positions
rhalf is 0.371809
rmax is 12.139229
Returning DM    Mass
Returning DM    Positions
12293.8093382 14700.5033977 11830.9886683 [  1.27115149e-02   2.18708866e-02   2.55220368e-02 ...,   5.75491696e+01
   5.75491903e+01   5.75492003e+01]
18.7846431946 34.7311124949


In [12]:
hnum = '948'
res = '_13'
ver = '11_13'
pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
hymass,hygmass,hysmass,hyx,irhalf, rmax= radpro(pathname,sname,hist,0,0,0)

pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

Returning DM    Mass
Returning DM    Positions
12343.4305731 13848.9611061 11202.8891286 [  1.43251064e-02   1.53530965e-02   2.39101610e-02 ...,   5.29716455e+01
   5.29717988e+01   5.29718006e+01]
Returning GAS   Mass
Returning STAR  Mass
Returning GAS   Positions
Returning STAR  Positions
rhalf is 0.643164
rmax is 5.159650
Returning DM    Mass
Returning DM    Positions
12342.6608232 13851.0208958 11199.3417098 [  1.46731634e-02   1.71551831e-02   1.78386700e-02 ...,   5.76337031e+01
   5.76337939e+01   5.76337979e+01]
28.7523744785 41.1819945455


In [13]:
hnum = '796'
res = '_13'
ver = '11_13'
pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
hymass,hygmass,hysmass,hyx,irhalf,rmax = radpro(pathname,sname,hist,0,0,0)

pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

Returning DM    Mass
Returning DM    Positions
11646.6386387 14159.4043145 12471.6837819 [  3.29538154e-02   3.66121951e-02   4.45153161e-02 ...,   5.51408022e+01
   5.51408119e+01   5.51408165e+01]
Returning GAS   Mass
Returning STAR  Mass
Returning GAS   Positions
Returning STAR  Positions
rhalf is 0.858267
rmax is 10.364631
Returning DM    Mass
Returning DM    Positions
11649.6442286 14160.0282536 12471.4758369 [  1.21427523e-02   1.41633283e-02   1.75021440e-02 ...,   6.02957190e+01
   6.02957226e+01   6.02957291e+01]
25.2618740731 35.1485900186


In [14]:
hnum = '897'
res = '_13'
ver = '11_13'
pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
hymass,hygmass,hysmass,hyx,irhalf,rmax = radpro(pathname,sname,hist,0,0,0)

pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

Returning DM    Mass
Returning DM    Positions
10940.2019471 14198.6648756 11711.9175472 [  1.03054963e-02   1.44256673e-02   2.05392647e-02 ...,   5.37886147e+01
   5.37886621e+01   5.37886726e+01]
Returning GAS   Mass
Returning STAR  Mass
Returning GAS   Positions
Returning STAR  Positions
rhalf is 0.396480
rmax is 9.697754
Returning DM    Mass
Returning DM    Positions
10938.6312057 14199.9534292 11717.1179197 [  2.13523007e-02   2.39225946e-02   2.52067227e-02 ...,   5.78309247e+01
   5.78309287e+01   5.78309513e+01]
16.973175155 35.5099241931


In [ ]:
#hnum = '1016'
#res = '_13'
#ver = '11_13'
#pathname = '/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/analysis/'%(hnum,res,ver)
#sname = "/nobackup/afitts/Gadget-2.0.7/production/mfm%s%s_giz%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,ver,i,i)
#hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
#hymass,hygmass,hysmass,hyx,irhalf,rmax = radpro(pathname,sname,hist,0,0,0)

#pathname = '/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/analysis/'%(hnum,res)
#sname = "/nobackup/afitts/Gadget-2.0.7/production/gizdm%s%s_raw_output/snapdir_%03d/snapshot_%03d"%(hnum,res,i,i)
#hist = np.genfromtxt(pathname+'halo%smerger_hist.txt'%hnum)
#dmomass,dmox = radpro(pathname,sname,hist,1,irhalf,rmax)
#plot_radpro(hnum,hymass,hygmass,hysmass,hyx,dmomass,dmox)

In [ ]:
ax1.set_ylabel("$v_{circ}$ (km/s)")
ax1.set_xlabel("$r_{1/2}$ (kpc)")
ax1.set_title("Vcirc v rhalf")
ax1.set_ylim(0,30)
ax1.legend(loc=4,prop={'size':8},ncol=1)
plt.show()
fig1.savefig('vcirc_vs_rhalf_1_14.pdf',transparent=True)